In [37]:
import gym
from random import randint
import numpy as np

import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

In [38]:
env = gym.make('MountainCar-v0', render_mode="rgb_array")

"""
Esta linea muestra la cantidad de acciones que tiene nuestro entorno
Para este caso el Action Space son 3, y son:
0 -> Acelerar a la izquierda
1 -> No hacer nada
2 -> Acelerar a la derecha
"""
env.action_space

Discrete(3)

In [39]:
"""
Esta linea muestra la cantidad de estados que tiene nuestro entorno
Para este caso el Observation Space son 2, y son:
0 -> Posicion de la montaña en el eje X (entre -1.2 y 0.6)
1 -> Velocidad del auto (entre -0.07 y 0.07)
"""
env.observation_space


Box([-1.2  -0.07], [0.6  0.07], (2,), float32)

In [40]:
def discretizar(valor):
    aux = ((valor - env.observation_space.low)/(env.observation_space.high - env.observation_space.low))*20
    return tuple(aux.astype(np.int32))

In [41]:
"""
env.reset() manda dos valores, el primero es el estado inicial y el segundo es un diccionario con información adicional
La idea para poder discretizar el estado es que el primer valor de la tupla que retorna env.reset() es el estado inicial
"""
estado,_ = env.reset()
discretizar(estado)

(7, 10)

In [42]:
"""
Ahora toca hacer la q_table, que es una tabla que contiene el valor de cada acción en cada estado
La q_table es una matriz de 20x20x3, donde 20 son los estados discretizados y 3 son las acciones posibles
Para partir se va a inicializar la q_table con valores aleatorios entre -1 y 1
"""
q_table = np.random.uniform(low=-1, high=1, size=[20, 20, 3])

## Fórmula de Q-Learning
La actualización de los valores Q sigue esta fórmula:


# New Q(s, a) = Q(s, a) + α [ R(s, a) + γ max Q(s', a') - Q(s, a) ]


Donde:

- **New Q(s, a)**: Nuevo valor Q para el estado `s` y acción `a`.
- **Q(s, a)**: Valor Q actual para el estado `s` y acción `a`.
- **α (alpha)**: Tasa de aprendizaje (Learning Rate).
- **R(s, a)**: Recompensa por tomar la acción `a` en el estado `s`.
- **γ (gamma)**: Tasa de descuento (Discount Rate), que mide la importancia de las recompensas futuras.
- **max Q(s', a')**: Máximo valor esperado de recompensa futura para el siguiente estado `s'` al tomar la mejor acción `a'`.

## Resumen visual:

| Elemento | Significado |
|:---|:---|
| **New Q(s, a)** | Nuevo valor Q para el estado y acción |
| **Q(s, a)** | Valores actuales de Q |
| **α** | Tasa de aprendizaje |
| **R(s, a)** | Recompensa obtenida por la acción en el estado |
| **γ** | Tasa de descuento |
| **max Q(s', a')** | Máxima recompensa futura esperada |


In [43]:
def graficar_q_table(q_table):
    mapa_acciones = np.argmax(q_table, axis=2)  # Elegimos la mejor acción para cada celda

    plt.figure(figsize=(10, 8))
    plt.imshow(mapa_acciones, cmap='viridis', origin='lower')
    plt.colorbar(ticks=[0,1,2], label='Acción: 0=Izquierda, 1=Nada, 2=Derecha')
    plt.title('Mapa de acciones preferidas')
    plt.xlabel('Velocidad Discretizada')
    plt.ylabel('Posición Discretizada')
    plt.show()

In [47]:
# env.reset(): Esto es para poder cambiar la posición de inicio
# alpha: tasa_aprendizaje = 0.1
# gamma:  tasa de descuento = 0.95 , si está próximo a cero busca las recompensa más cercanas, si está próximo a uno busca las recompensas más lejanas -> confianza
# epsilon: tasa de exploración = 0.1
alpha = 0.1
gamma = 0.95
epsilon = 0.7
iteraciones = 5000
lista_recompensas = []

# Inicializar grafico de la Q-table
plt.ion()  # Modo interactivo

fig, ax = plt.subplots(figsize=(8, 6))
img = ax.imshow(np.argmax(q_table, axis=2), cmap='viridis', origin='lower')
cbar = plt.colorbar(img, ax=ax, ticks=[0, 1, 2])
cbar.ax.set_yticklabels(['Izquierda', 'Nada', 'Derecha'])
ax.set_title('Mapa de acciones preferidas (entrenando)')
ax.set_xlabel('Velocidad Discretizada')
ax.set_ylabel('Posición Discretizada')

for iteracion in range(iteraciones):
    estado,_ = env.reset()
    estado = discretizar(estado)
    final = False
    recompensa_total = 0
    while not final:
        """
        # 20% que sea aleatoria y 80% que sea la mejor acción
        if randint(0, 10) > 2:
            accion = np.argmax(q_table[estado])
        else:
            accion = randint(0, 2)
        """
        if np.random.uniform(0,1) < epsilon:
            accion = np.random.randint(0,3) # Exploracion -> accion aleatoria
        else:
            accion = np.argmax(q_table[estado]) # Mejor accion
        nuevo_estado, recompensa, final, truncado, info = env.step(accion)
        # Esta es la fórmula de Q-Learning, que representa
        q_table[estado][accion] = q_table[estado][accion] + alpha * (recompensa + gamma * np.max(q_table[discretizar(nuevo_estado)]) - q_table[estado][accion])
        estado = discretizar(nuevo_estado)
        recompensa_total += recompensa


        if (iteracion+1) % 1000 == 0:
            env.render()

    lista_recompensas.append(recompensa_total)
    # Decaimiento de epsilon
    epsilon = max(0.01, epsilon * 0.995)

    # Mostrar progreso
    if (iteracion+1) % 100 == 0:
        print(f"Iteración: {iteracion+1}, Recompensa total: {recompensa_total}")
        print(f"Media de las recompensas: {np.mean(lista_recompensas[-100:])}")

        img.set_data(np.argmax(q_table, axis=2))
        ax.set_title(f'Mapa de acciones preferidas (Iteración {iteracion+1})')
        fig.canvas.draw()
        fig.canvas.flush_events()

env.close()
# env.close() cierra la ventana de renderizado

plt.ioff() # Desactiva el modo interactivo
plt.show()

Iteración: 100, Recompensa total: -249.0
Media de las recompensas: -366.35
Iteración: 200, Recompensa total: -158.0
Media de las recompensas: -223.71
Iteración: 300, Recompensa total: -137.0
Media de las recompensas: -181.25
Iteración: 400, Recompensa total: -213.0
Media de las recompensas: -165.59
Iteración: 500, Recompensa total: -143.0
Media de las recompensas: -158.3
Iteración: 600, Recompensa total: -222.0
Media de las recompensas: -153.82
Iteración: 700, Recompensa total: -152.0
Media de las recompensas: -162.73
Iteración: 800, Recompensa total: -153.0
Media de las recompensas: -182.06
Iteración: 900, Recompensa total: -151.0
Media de las recompensas: -158.6
Iteración: 1000, Recompensa total: -170.0
Media de las recompensas: -171.69
Iteración: 1100, Recompensa total: -153.0
Media de las recompensas: -141.05
Iteración: 1200, Recompensa total: -159.0
Media de las recompensas: -148.45
Iteración: 1300, Recompensa total: -157.0
Media de las recompensas: -161.36
Iteración: 1400, Recomp

In [ ]:
ventana = 100  # Tamaño de la ventana del promedio
promedio_movil = np.convolve(lista_recompensas, np.ones(ventana)/ventana, mode='valid')

plt.figure(figsize=(10, 6))
plt.plot(promedio_movil, label=f'Promedio móvil (ventana={ventana})', color='royalblue', linewidth=2)
plt.title('Promedio Móvil de Recompensas', fontsize=16)
plt.xlabel('Episodios', fontsize=14)
plt.ylabel('Recompensa Promedio', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()
plt.show()

In [45]:
plt.plot(np.convolve(lista_recompensas, np.ones(100) / 100, mode='valid'))
plt.title('Promedio móvil de recompensas (ventana=100)')
plt.xlabel('Episodios')
plt.ylabel('Recompensa promedio')
plt.grid()
plt.show()

In [46]:
graficar_q_table(q_table)